# 우분투 14.04에 NVIDA CUDA 드라이버설치 

## 원본자료 :  http://docs.nvidia.com/cuda/cuda-getting-started-guide-for-linux/index.html#pre-installation-actions

## 01. 시스템 요구사항 
- CUDA를 지원하는 그래픽카드
- gcc 컴파일러와 toolchain을 설치된 CUDA운영이 가능한 리눅스 버전
   - sudo apt-get install build-essential
   - sudo apt-get install linux-headers-`uname -r`
- NVIDIA CUDA Toolkit ( http://developer.nvidia.com/cuda-downloads )

## 02. 설치 준비 작업
- CUDA를 지원하는 그래픽카드가 설치되어 있는지 확인
- CUDA운영이 가능한 리눅스 버전인지 확인
- gcc가 설치되어 있는지 확인
- NVIDIA CUDA Toolkit 다운로드
- 설치시 장애 처리 방법( uninstall 방법 )

### CUDA를 지원하는 그래픽카드가 설치되어 있는지 확인
```
[deeplearning@deep01 ~]$ lspci | grep -i nvidia
02:00.0 VGA compatible controller: NVIDIA Corporation Device 1401 (rev a1)
02:00.1 Audio device: NVIDIA Corporation Device 0fba (rev a1)
```

In [3]:
! lspci | grep -i nvidia

### CUDA운영이 가능한 리눅스 버전인지 확인

In [4]:
! uname -m && cat /etc/*release

x86_64
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=14.04
DISTRIB_CODENAME=trusty
DISTRIB_DESCRIPTION="Ubuntu 14.04.3 LTS"
NAME="Ubuntu"
VERSION="14.04.3 LTS, Trusty Tahr"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 14.04.3 LTS"
VERSION_ID="14.04"
HOME_URL="http://www.ubuntu.com/"
SUPPORT_URL="http://help.ubuntu.com/"
BUG_REPORT_URL="http://bugs.launchpad.net/ubuntu/"


### gcc가 설치되어 있는지 확인

In [6]:
! gcc --version

gcc (Ubuntu 4.8.4-2ubuntu1~14.04) 4.8.4
Copyright (C) 2013 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



### NVIDIA CUDA Toolkit 다운로드

- 최신버전( 7.5 ) : http://developer.nvidia.com/cuda-downloads 
- 7.0버전 : https://developer.nvidia.com/cuda-toolkit-70
- 6.5버전 : https://developer.nvidia.com/cuda-toolkit-65
- 6.5버전의 GeForce GTX980 또는 GTX970 용 : https://developer.nvidia.com/cuda-downloads-geforce-gtx9xx 

In [ ]:
! wget http://developer.download.nvidia.com/compute/cuda/7_0/Prod/local_installers/cuda_7.0.28_linux.run

### 설치시 장애 처리 방법
- runfile 로 설치했을때 uninstall 방법
```
sudo /usr/local/cuda-X.Y/bin/uninstall_cuda_X.Y.pl
sudo /usr/bin/nvidia-uninstall
```

- RPM/Deb 로 설치했을때 uninstall 방법
```
$ sudo apt-get --purge remove <package_name> # Ubuntu 
$ sudo yum remove <package_name> # Fedora/Redhat/CentOS 
$ sudo zypper remove <package_name> # OpenSUSE/SLES
```

## 03. NVIDIA CUDA Toolkit 설치

### 우분투를 텍스트모드 <-> X윈도우모드로 전환하는 스크립트 작성
```
sudo cp -n /etc/default/grub /etc/default/grub.xmode
sudo vi /etc/default/grub
#GRUB_CMDLINE_LINUX_DEFAULT="quiet splash"
GRUB_CMDLINE_LINUX="text"
#GRUB_TERMINAL=console
sudo cp -n /etc/default/grub /etc/default/grub.tmode

echo 'sudo cp /etc/default/grub.tmode /etc/default/grub && sudo update-grub' >> t-mode.sh
echo 'sudo cp /etc/default/grub.xmode /etc/default/grub && sudo update-grub' >> x-mode.sh
chmod +x *.sh
```
- 텍스트모드 전환 : sudo ./t-mode.sh && sudo reboot 
- X윈도우모드 전환 :sudo  ./x-mode.sh && sudo reboot 

### 우분투 서버에 장착된 그래픽카드 드라이버를 다운로드하고 설치함
- cuda 6.5설치시만 필요함.
- http://www.nvidia.co.kr/Download/Find.aspx?lang=kr
- 리눅스용 그래픽카드 드라이버 설치시 root권한( sudo )이 필요하고,  텍스트모드로 부팅되어야 함.

### NVIDIA CUDA Toolkit 설치
- root권한( sudo )이 필요하고, 텍스트모드로 부팅되어야 함.
- Network Installer( .rpm, .deb)
- Local Package Installer( .rpm, .deb)	
- Runfile Installer( .run )

####  Network Installer
```
sudo dpkg -i cuda-repo-<distro>_<version>_<architecture>.deb
sudo apt-get update
sudo apt-get install cuda
```

#### Runfile Installer
```
# cuda 7 일때
sudo ./cuda_7.0.28_linux.run

# cuda 6.5 일때
sudo ./cuda_6.5.14_linux_64.run --kernel-source-path=/usr/src/linux-headers-`uname -r`/
```

#### Library 등록
```
# cuda 7 일때
echo 'export PATH=/usr/local/cuda-7.0/bin:$PATH' >> ~/.bashrc 
echo 'export LD_LIBRARY_PATH=/usr/local/cuda-7.0/lib64:$LD_LIBRARY_PATH' >> ~/.bashrc
source ~/.bashrc

# cuda 6.5 일때
echo 'export PATH=/usr/local/cuda/bin:$PATH' >> ~/.bashrc
echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64:/usr/local/lib' >> ~/.bashrc
source ~/.bashrc
```

## 04. 설치 확인

### 드라이브 버전 확인
```
[deeplearning@deep01 ~]$ cat /proc/driver/nvidia/version
NVRM version: NVIDIA UNIX x86_64 Kernel Module  346.46  Tue Feb 17 17:56:08 PST 2015
GCC version:  gcc version 4.8.4 (Ubuntu 4.8.4-2ubuntu1~14.04)
```

In [8]:
! cat /proc/driver/nvidia/version

cat: /proc/driver/nvidia/version: 그런 파일이나 디렉터리가 없습니다


### 예제 컴파일
```
nvcc -V    # cuda 컴파일러 버전 확인
cd ~/NVIDIA_CUDA-7.0_Samples
make
0_Simple/simplePrintf/simplePrintf
0_Simple/matrixMul/matrixMul
0_Simple/matrixMulCUBLAS/matrixMulCUBLAS
```

## 05. Theano에서 GPU와 CPU 사용법
```
# CPU
THEANO_FLAGS=mode=FAST_RUN,device=cpu,floatX=float32 python check1.py

# GPU
THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python check1.py
```


In [1]:
# CPU
! THEANO_FLAGS=mode=FAST_RUN,device=cpu,floatX=float32

In [5]:
# GPU
! THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32

In [6]:
# check1.py
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in xrange(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 6.359674 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the cpu


```
(theano-env)[deeplearning@deep01 misc]$ THEANO_FLAGS=mode=FAST_RUN,device=cpu,floatX=float32 python check1.py
[Elemwise{exp,no_inplace}(<TensorType(float32, vector)>)]
Looping 1000 times took 3.268355 seconds
Result is [ 1.23178029  1.61879337  1.52278066 ...,  2.20771813  2.29967761
  1.62323284]
Used the cpu
(theano-env)[deeplearning@deep01 misc]$ THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python check1.py
Using gpu device 0: GeForce GTX 960
[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.568380 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu

```
